In [2]:
import argparse
import cv2
import os
import numpy as np
import random
import math
import sys
import time
import matplotlib.pyplot as plt 
import shutil

NAMES = [ 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
         'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
         'U', 'V', 'W', 'X', 'Y', 'Z']

cube_list = []
cube_names = []
background_list = []
background_names = []

NUM_TOTAL_VARIATIONS = 100000
SAMPLES_PER_IMAGE = 3000
SAMPLE = True

In [ ]:


def load_images_from_folder(folder, images, names):
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
            images.append(img)
            names.append(filename)
            #plt.imshow( img)
            #plt.show()
            
    return images

def rotation(image, angleInDegrees):
    h, w = image.shape[:2]
    img_c = (w / 2, h / 2)

    rot = cv2.getRotationMatrix2D(img_c, 360-angleInDegrees, 1)

    rad = math.radians(angleInDegrees)
    sin = math.sin(rad)
    cos = math.cos(rad)
    b_w = int((h * abs(sin)) + (w * abs(cos)))
    b_h = int((h * abs(cos)) + (w * abs(sin)))

    rot[0, 2] += ((b_w / 2) - img_c[0])
    rot[1, 2] += ((b_h / 2) - img_c[1])

    outImg = cv2.warpAffine(image, rot, (b_w, b_h), flags=cv2.INTER_LINEAR)
    return outImg

#yolo annotation format, normal: [class, x_m, y_m, d_x, d_y], here: +angle 
def writeYOLOAnnotation(textfile, idx, bb, angle):
    name_index = NAMES.index(os.path.splitext(cube_names[idx])[0])          
    with open(textfile, 'a') as the_file:
        the_file.write(str(name_index))
        the_file.write(' ')
        the_file.write(str(bb[0]))
        the_file.write(' ')
        the_file.write(str(bb[1]))
        the_file.write(' ')
        the_file.write(str(bb[2]))
        the_file.write(' ')
        the_file.write(str(bb[3]))
        the_file.write(' ')
        the_file.write(str(angle))
        the_file.write('\n')

def writeYOLOAnnotation_without_rot(textfile, idx, bb):
    name_index = NAMES.index(os.path.splitext(cube_names[idx])[0])          
    with open(textfile, 'a') as the_file:
        the_file.write(str(name_index))
        the_file.write(' ')
        the_file.write(str(bb[0]))
        the_file.write(' ')
        the_file.write(str(bb[1]))
        the_file.write(' ')
        the_file.write(str(bb[2]))
        the_file.write(' ')
        the_file.write(str(bb[3]))
        the_file.write(' ')
        the_file.write('\n')

def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

def updt(total, progress):
    """
    Displays or updates a console progress bar.

    Original source: https://stackoverflow.com/a/15860757/1391441
    """
    barLength, status = 20, ""
    progress = float(progress) / float(total)
    if progress >= 1.:
        progress, status = 1, "\r\n"
    block = int(round(barLength * progress))
    text = "\r[{}] {:.0f}% {}".format(
        "#" * block + "-" * (barLength - block), round(progress * 100, 0),
        status)
    sys.stdout.write(text)
    sys.stdout.flush()

In [6]:
folder_path = "/Users/paulschulz/Cube_detection/Dataset_home/base/"
images_path = "/Users/paulschulz/Cube_detection/Dataset_home/base/images"
labels_path = "/Users/paulschulz/Cube_detection/Dataset_home/base/labels"

cube_path = "/Users/paulschulz/Cube_detection/Dataset_home/cubes"
background_path = '/Users/paulschulz/Cube_detection/Dataset_home/backgrounds'



if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)

if os.path.exists(labels_path):
    shutil.rmtree(labels_path)
os.mkdir(labels_path)

In [ ]:




if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)
    

if os.path.exists(labels_path):
    shutil.rmtree(labels_path)
os.mkdir(labels_path)